In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('train.csv')

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df['text'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [9]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
df=df.dropna()

In [11]:
X = df.drop('label',axis=1)
y= df['label']
X.shape

(18285, 4)

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
voc_size=5000

In [14]:
messages=X.copy()

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
#nltk.download('stopwords')

In [18]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [19]:
X.shape

(18285, 4)

In [20]:
X = X[~X['title'].str.isnumeric()]


In [21]:
X.shape

(18285, 4)

In [22]:
print(messages.columns)


Index(['index', 'id', 'title', 'author', 'text'], dtype='object')


In [23]:
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1:5]

[[40, 1422, 64, 3439, 3654, 1161, 644],
 [2408, 3341, 757, 2695],
 [3163, 4442, 2121, 41, 3275, 2142],
 [3478, 3654, 3312, 1631, 423, 1769, 3654, 19, 1281, 4361]]

One-hot encoding is a simple way to represent words numerically, but it doesn't capture semantic similarities between words.

<h3>Using Word Embeddings and LSTM
<h4>Add Word Embeddings just add zero in the beginning or the End


In [25]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs[1:10])

[[  40 1422   64 3439 3654 1161  644    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [2408 3341  757 2695    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [3163 4442 2121   41 3275 2142    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [3478 3654 3312 1631  423 1769 3654   19 1281 4361    0    0    0    0
     0    0    0    0    0    0]
 [ 585 1507 2824 3664 2569 4149  581  436   18 1780  915 3677 3942  380
   644    0    0    0    0    0]
 [4309 2203 3541 1688   91 4871 1506 2261 1578 2012 4360    0    0    0
     0    0    0    0    0    0]
 [2341 3025 1264 2757  979 4950 4149   56 1578 2012 4360    0    0    0
     0    0    0    0    0    0]
 [ 348 3702  922 4350   76  446  286 3834 4149 1238    0    0    0    0
     0    0    0    0    0    0]
 [ 589 1935  261 1250 1925 1945 2159 1106    0    0    0    0    0    0
     0    0    0    0    0    0]]


In [26]:
embedded_vector_features=40
model=Sequential()

<h4>Applying LSTM on the Data 

In [27]:
model.add(Embedding(voc_size,embedded_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# new_learning_rate = 0.001
# model.compile(optimizer=SGD(learning_rate=new_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [28]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [29]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [31]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10


192/192 [==============================] - 13s 39ms/step - loss: 0.3054 - accuracy: 0.8489 - val_loss: 0.1992 - val_accuracy: 0.9157
Epoch 2/10
192/192 [==============================] - 5s 27ms/step - loss: 0.1426 - accuracy: 0.9455 - val_loss: 0.2012 - val_accuracy: 0.9183
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0927 - accuracy: 0.9675 - val_loss: 0.2464 - val_accuracy: 0.9155
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0649 - accuracy: 0.9789 - val_loss: 0.2666 - val_accuracy: 0.9110
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0528 - accuracy: 0.9833 - val_loss: 0.2861 - val_accuracy: 0.9148
Epoch 6/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0393 - accuracy: 0.9875 - val_loss: 0.3474 - val_accuracy: 0.9128
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0267 - accuracy: 0.9923 - val_loss: 0.3642 - val_accuracy: 0.9118
Epoch 8/1

<h3>Adding Dropout Layer to the Model

In [32]:
from tensorflow.keras.layers import Dropout
import tensorflow as tf
## Creating model
embedding_vector_features=40
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model2.add(Dropout(0.3))
model2.add(LSTM(100))
model2.add(Dropout(0.3))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [33]:
early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience= 20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [34]:
### Finally Training
model_history=model2.fit(X_train,y_train,validation_split=0.33,batch_size=32,epochs=30,callbacks=early_stopping)

Epoch 1/30
257/257 [==============================] - 12s 31ms/step - loss: 0.3096 - accuracy: 0.8531 - val_loss: 0.1968 - val_accuracy: 0.9159
Epoch 2/30
257/257 [==============================] - 7s 29ms/step - loss: 0.1371 - accuracy: 0.9491 - val_loss: 0.2295 - val_accuracy: 0.9048
Epoch 3/30
257/257 [==============================] - 7s 28ms/step - loss: 0.0890 - accuracy: 0.9701 - val_loss: 0.2436 - val_accuracy: 0.9129
Epoch 4/30
257/257 [==============================] - 7s 27ms/step - loss: 0.0613 - accuracy: 0.9794 - val_loss: 0.2994 - val_accuracy: 0.9040
Epoch 5/30
257/257 [==============================] - 8s 30ms/step - loss: 0.0505 - accuracy: 0.9831 - val_loss: 0.3160 - val_accuracy: 0.9100
Epoch 6/30
257/257 [==============================] - 8s 31ms/step - loss: 0.0350 - accuracy: 0.9899 - val_loss: 0.3831 - val_accuracy: 0.8998
Epoch 7/30
257/257 [==============================] - 8s 31ms/step - loss: 0.0338 - accuracy: 0.9889 - val_loss: 0.4588 - val_accuracy: 0.890

In [35]:
y_pred=model2.predict(X_test)

189/189 [==============================] - 1s 5ms/step


In [36]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve
from sklearn.metrics import confusion_matrix

In [37]:
confusion_matrix(y_test,y_pred)

array([[3111,  308],
       [ 293, 2323]], dtype=int64)

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      3419
           1       0.88      0.89      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035



<h3>Using Tensorflow's Word2vec

<h3>Additional Tasks to perform on this project
<h5>->In the Neural Network try using different Optimizers along with changing learning rate to see its effect on accuracy
<h5>->Increase the number of epochs and apply a limit to it to get the best accuracy  
<h5>->Also use pre-made word2vec of Tensorflow